In [0]:
!pip install bert-for-tf2

In [0]:
!pip install tqdm

In [0]:
!nvidia-smi


In [0]:
!pip install gdown
!pip install tensorflow_text

In [0]:
!pip install --upgrade grpcio >> /dev/null



In [0]:

import os
import math
import datetime

from tqdm import tqdm

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras

import bert
from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib import rc

from sklearn.metrics import confusion_matrix, classification_report

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [0]:
!gdown --id 1OlcvGWReJMuyYQuOZm149vHWwPtlboR6 --output train.csv
!gdown --id 1Oi5cRlTybuIF2Fl5Bfsr-KkqrXrdt77w --output valid.csv
!gdown --id 1ep9H6-HvhB4utJRLVcLzieWNUSG3P_uF --output test.csv

In [0]:
train = pd.read_csv("train.csv")
valid = pd.read_csv("valid.csv")
test = pd.read_csv("test.csv")

In [0]:
train = train.append(valid).reset_index(drop=True)



In [0]:
train.head()

In [0]:
chart = sns.countplot(train.intent, palette=HAPPY_COLORS_PALETTE)
plt.title("Number of texts per intent")
chart.set_xticklabels(chart.get_xticklabels(), rotation=30, horizontalalignment='right');

In [0]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip


In [0]:
!unzip uncased_L-12_H-768_A-12.zip


In [0]:
os.makedirs("model", exist_ok=True)


In [0]:
!mv uncased_L-12_H-768_A-12/ model


In [0]:
bert_model_name="uncased_L-12_H-768_A-12"

bert_ckpt_dir = os.path.join("model/", bert_model_name)
bert_ckpt_file = os.path.join(bert_ckpt_dir, "bert_model.ckpt")
bert_config_file = os.path.join(bert_ckpt_dir, "bert_config.json")

In [0]:
train.head()

In [0]:
class IntentDetectionData:
  DATA_COLUMN='text'
  LABEL_COLUMN='intent'

  def __init__(self,train,test,tokenizer: FullTokenizer,classes,max_seq_len=192):
    self.tokenizer=tokenizer
    self.max_seq_len=0
    self.classes=classes

    ((self.train_x,self.train_y),(self.test_x,self.test_y)) = map(self._prepare,[train,test])
    self.max_seq_len = min(self.max_seq_len, max_seq_len)
    self.train_x,self.test_x = map(self._pad,[self.train_x,self.test_x])


  def _prepare(self,df):
    x ,y =[],[]
    for _,row in tqdm(df.iterrows()):
      text,label=row[IntentDetectionData.DATA_COLUMN], row[IntentDetectionData.LABEL_COLUMN]
      tokens = self.tokenizer.tokenize(text)
      tokens = ["[CLS]"] + tokens + ["[SEP]"]
      token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
      self.max_seq_len=max(self.max_seq_len,len(token_ids))
      x.append(token_ids)
      y.append(self.classes.index(label))
    return np.array(x),np.array(y)
  
  def _pad(self,ids):
    x=[]
    for input_ids in ids:
      cut_point=min(len(input_ids),self.max_seq_len-2)
      input_ids=input_ids[:cut_point]
      input_ids = input_ids +[0]*(self.max_seq_len - len(input_ids))
      x.append(input_ids)

    return np.array(x)









In [0]:
tokenizer = FullTokenizer(vocab_file=os.path.join(bert_ckpt_dir,'vocab.txt'))

In [0]:
tokenizer.tokenize("I can't wait to visit bulgaria again!")

In [0]:
def create_model(max_seq_len,bert_config_file,bert_ckpt_file):
  with tf.io.gfile.GFile(bert_config_file,'r') as reader:
    bc = StockBertConfig.from_json_string(reader.read())
    bert_params=map_stock_config_to_params(bc)
    bert_params.adapter_size = None
    bert = BertModelLayer.from_params(bert_params, name = 'bert')

  input_ids = keras.layers.Input(shape=(max_seq_len,),dtype='int32',name='input_ids')
  bert_output=bert(input_ids)

  print('bert shape',bert_output.shape)

  cls_out = keras.layers.Lambda(lambda seq : seq[:,0,:])(bert_output)
  cls_out= keras.layers.Dropout(0.5)(cls_out)
  logits = keras.layers.Dense(units=768, activation='tanh')(cls_out)
  logits = keras.layers.Dropout(0.5)(logits)
  logits = keras.layers.Dense(units=len(classes), activation='softmax')(logits)
  model=keras.Model(inputs=input_ids, outputs=logits)
  model.build(input_shape=(None, max_seq_len))

  load_stock_weights(bert, bert_ckpt_file)
        
  return model


In [0]:
classes = train.intent.unique().tolist()
data =IntentDetectionData(train,test,tokenizer,classes,max_seq_len=128)

In [0]:
model=create_model(data.max_seq_len,bert_config_file,bert_ckpt_file)

In [0]:
data.train_x.shape

In [0]:
data.train_y.shape 

In [0]:
model.summary()

In [0]:
model.compile(optimizer=keras.optimizers.Adam(1e-5),loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=[keras.metrics.SparseCategoricalCrossentropy(name='acc')])

In [0]:
history = model.fit(
  x=data.train_x, 
  y=data.train_y,
  validation_split=0.1,
  batch_size=16,
  shuffle=True,
  epochs=5)